In [1]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import time
import numpy as np
import ast

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener el valor de OPENAI_API_KEY
api_key = os.getenv("OPENAI_API_KEY")

gpt_client = OpenAI(
    api_key = api_key,
)

# Carga el archivo Parquet
# data = pd.read_parquet("../../supermercados/plazavea/plazavea.parquet")
# data = pd.read_parquet("../../supermercados/vivanda/vivanda.parquet")
data = pd.read_parquet("../../supermercados/wong/wong.parquet")
# data = pd.read_parquet("../productos_supermercados.parquet")

In [2]:
data.sample(n=5)

,product_name,link,descripcion_producto,price,source,categories
28355,Vela Krea Redonda Blanca Navidad 2024,https://www.wong.pe/vela-krea-redonda-blanca-n...,Wong,5.60,Wong,"[Navidad, Decoración Navideña, Velas Navideñas]"
22199,Libro Super Color +4 Amarillo,https://www.wong.pe/libro-super-color-4-amaril...,Wong,14.99,Wong,"[Libros y Librería, Libros, Libros para Colore..."
15158,Botella Play Pc 1.4 Lt Extraíble Blue,https://www.wong.pe/botella-play-pc-1-4-lt-ext...,Wong,34.99,Wong,"[Hogar y Bazar, Cocina, Tomatodos]"
6358,Collar Minnie Ribbon Pink,https://www.wong.pe/collar-minnie-ribbon-pink-...,Wong,24.90,Wong,"[Mascotas, Para Perros, Artículos de Transporte]"
13818,Silla Atlante Decohome Verde,https://www.wong.pe/silla-atlante-decohome-ver...,Wong,713.82,Wong,"[Hogar y Bazar, Muebles, Sillas y Butacas]"


In [3]:
# Función para obtener embeddings
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    response = gpt_client.embeddings.create(input = [text], model=model).data[0].embedding
    return response

# Generar embeddings y agregarlos como una nueva columna
data["embedding"] = data.apply(
    lambda row: get_embedding(
        f"Producto: {row['product_name']}. Descripción: {row['descripcion_producto']}. Categorías: {' ,'.join(row['categories'])}"
    ), 
    axis=1
)

In [4]:
data['categories_txt'] = data['categories'].apply(lambda x: ', '.join(ast.literal_eval(x) if isinstance(x, str) else x))

# Guarda los embeddings para uso posterior
data.to_parquet("productos_con_embeddings.parquet")
# data.to_parquet("wong_con_embeddings.parquet")